<a href="https://colab.research.google.com/github/carakhan15/doing-things/blob/master/Surprise_Netflix_Recommendation_System_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ingest

### Mount GDrive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Listing Files

In [0]:
import os;os.listdir("/content/gdrive/My Drive/ML")

['hi.gdoc',
 'kaggle.json',
 'wine.csv',
 'house-prices-advanced-regression-techniques',
 'County_time_series.csv',
 'City_time_series.csv',
 'combined_data_1.txt.zip',
 'combined_data_1.txt.zip (Unzipped Files)',
 'Archive.zip',
 'HW7']

In [0]:
!pip install kaggle

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!cp /content/gdrive/My\ Drive/ML/kaggle.json ~/.kaggle/kaggle.json

Metadata

In [0]:
import pandas as pd
#!kaggle datasets download -d netflix-inc/netflix-prize-data
#!ls -l /content
movie_metadata=pd.read_csv("movie_metadata.csv", low_memory=False)[['original_title', 'overview', 'vote_count']].set_index('original_title').dropna()
# Remove the long tail of rarly rated moves
movie_metadata = movie_metadata[movie_metadata['vote_count']>10].drop('vote_count', axis=1)

print('Shape Movie-Metadata:\t{}'.format(movie_metadata.shape))
movie_metadata.sample(5)


Shape Movie-Metadata:	(21604, 1)


,overview
original_title,
"Vernon, Florida",Early Errol Morris documentary intersplices ra...
Άλπεις,"A nurse, a paramedic, a gymnast and her coach ..."
笑拳怪招,"Jackie Chan is a youngster, living in a remote..."
Mere Brother Ki Dulhan,London-based Luv Agnihotri decides to end his ...
Life as a House,"When a man is diagnosed with terminal cancer, ..."


### Training Data

In [0]:
!unzip /content/gdrive/'My Drive'/ML/'combined_data_1.txt.zip'

Archive:  /content/gdrive/My Drive/ML/combined_data_1.txt.zip
replace combined_data_1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
import pandas as pd
df_raw = pd.read_csv('../content/combined_data_1.txt', header=None, names=['User', 'Rating', 'Date'], usecols=[0, 1, 2])

In [0]:
df_raw.head(3)

,User,Rating,Date
0,1:,NaN,NaN
1,1488844,3.0,2005-09-06
2,822109,5.0,2005-05-13


#### Load Movie Titles

In [0]:
!ls -l /content/gdrive/My\ Drive/__MACOSX/

In [0]:
  !unzip /content/gdrive/'My Drive'/ML/'Archive.zip'

Movie Titles

In [0]:
movie_titles = pd.read_csv("movie_title_s.csv",  
                           encoding = 'ISO-8859-1', 
                           header = None, 
                           names = ['Id', 'Year', 'Name']).set_index('Id')
print('Shape Movie-Titles:\t{}'.format(movie_titles.shape))
movie_titles.sample(5)

Shape Movie-Titles:	(17770, 2)


,Year,Name
Id,,
10575,1962.0,The Twilight Zone: Vol. 6
952,1940.0,My Favorite Wife
2,2004.0,Isle of Man TT 2004 Review
17643,1984.0,Supergirl
7509,2002.0,Die Another Day


#### Preprocessing

In [0]:
from collections import deque

tmp_movies = df_raw[df_raw['Rating'].isna()]['User'].reset_index()
movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]

# Shift the movie_indices by one to get start and endpoints of all movies
shifted_movie_indices = deque(movie_indices)
shifted_movie_indices.rotate(-1)


# Gather all dataframes
user_data = []

# Iterate over all movies
for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
    
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = df_raw.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = df_raw.loc[df_id_1+1:].copy()
        
    # Create movie_id column
    tmp_df['Movie'] = movie_id
    
    # Append dataframe to list
    user_data.append(tmp_df)

# Combine all dataframes
df_main = pd.concat(user_data)
del user_data, df_raw, tmp_movies, tmp_df, shifted_movie_indices, movie_indices, df_id_1, movie_id, df_id_2, next_movie_id
print('Shape User-Ratings:\t{}'.format(df_main.shape))
df_main.sample(5)

Shape User-Ratings:	(24053764, 4)


,User,Rating,Date,Movie
23776684,2463504,4.0,2005-01-03,4432
8668240,1540611,4.0,2004-02-29,1728
1994211,999532,5.0,2005-05-02,361
13752173,2051278,5.0,2005-02-11,2617
10520727,1102433,5.0,2005-03-12,2045


In [0]:
df_main.sample(5)

,User,Rating,Date,Movie
1156776,2331185,1.0,2005-10-12,267
4660266,2306038,4.0,2004-12-08,907
16998824,1270817,4.0,2005-04-29,3282
18139151,1181168,5.0,2005-09-25,3446
19469657,1789629,5.0,2005-04-23,3713


In [0]:
df = df_main.sample(100000)

In [0]:
df.head()

,User,Rating,Date,Movie
3185670,858799,1.0,2004-01-28,582
12496851,671728,3.0,2005-09-23,2391
14678696,2555688,5.0,2005-04-30,2831
9066718,1081461,4.0,2003-07-08,1798
7979427,1631810,5.0,2005-10-15,1595


## EDA

In [0]:
import pandas as pd

# To do linear algebra
import numpy as np

# To create plots
import matplotlib.pyplot as plt

# To create interactive plots
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)


## Surprise!

In [0]:
!pip install -q surprise

In [0]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import time
import datetime
import random

import numpy as np
import six
from tabulate import tabulate

from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering

# The algorithms to cross-validate
classes = (SVD, SVDpp, NMF, SlopeOne, KNNBasic, KNNWithMeans, KNNBaseline,
           CoClustering, BaselineOnly, NormalPredictor)

# ugly dict to map algo names and datasets to their markdown links in the table
stable = 'http://surprise.readthedocs.io/en/stable/'
LINK = {'SVD': '[{}]({})'.format('SVD',
                                 stable +
                                 'matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD'),
        'SVDpp': '[{}]({})'.format('SVD++',
                                   stable +
                                   'matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVDpp'),
        'NMF': '[{}]({})'.format('NMF',
                                 stable +
                                 'matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.NMF'),
        'SlopeOne': '[{}]({})'.format('Slope One',
                                      stable +
                                      'slope_one.html#surprise.prediction_algorithms.slope_one.SlopeOne'),
        'KNNBasic': '[{}]({})'.format('k-NN',
                                      stable +
                                      'knn_inspired.html#surprise.prediction_algorithms.knns.KNNBasic'),
        'KNNWithMeans': '[{}]({})'.format('Centered k-NN',
                                          stable +
                                          'knn_inspired.html#surprise.prediction_algorithms.knns.KNNWithMeans'),
        'KNNBaseline': '[{}]({})'.format('k-NN Baseline',
                                         stable +
                                         'knn_inspired.html#surprise.prediction_algorithms.knns.KNNBaseline'),
        'CoClustering': '[{}]({})'.format('Co-Clustering',
                                          stable +
                                          'co_clustering.html#surprise.prediction_algorithms.co_clustering.CoClustering'),
        'BaselineOnly': '[{}]({})'.format('Baseline',
                                          stable +
                                          'basic_algorithms.html#surprise.prediction_algorithms.baseline_only.BaselineOnly'),
        'NormalPredictor': '[{}]({})'.format('Random',
                                             stable +
                                             'basic_algorithms.html#surprise.prediction_algorithms.random_pred.NormalPredictor'),
        'ml-100k': '[{}]({})'.format('Movielens 100k',
                                     'http://grouplens.org/datasets/movielens/100k'),
        'ml-1m': '[{}]({})'.format('Movielens 1M',
                                   'http://grouplens.org/datasets/movielens/1m'),
        }


In [0]:
np.random.seed(0)
random.seed(0)

In [0]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import pandas as pd

from surprise import NormalPredictor
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df[['User', 'Movie', 'Rating']], reader=reader)

### Testing Models

In [0]:
cross_validate(NormalPredictor(), data, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4720  1.4625  1.4702  1.4673  1.4705  1.4685  0.0034  
MAE (testset)     1.1831  1.1725  1.1756  1.1735  1.1807  1.1771  0.0041  
Fit time          0.16    0.21    0.18    0.18    0.18    0.18    0.01    
Test time         0.43    0.41    0.37    0.17    0.17    0.31    0.12    


{'fit_time': (0.1645946502685547,
  0.20882320404052734,
  0.17647075653076172,
  0.17935395240783691,
  0.17812585830688477),
 'test_mae': array([1.18312268, 1.17248056, 1.17560827, 1.17351383, 1.18071899]),
 'test_rmse': array([1.47198256, 1.46253836, 1.47023486, 1.46732318, 1.47053337]),
 'test_time': (0.43105101585388184,
  0.40632128715515137,
  0.36931395530700684,
  0.1703650951385498,
  0.168778657913208)}

In [0]:
cross_validate(SVD(), data, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0181  1.0251  1.0246  1.0234  1.0195  1.0221  0.0028  
MAE (testset)     0.8177  0.8226  0.8225  0.8253  0.8140  0.8204  0.0040  
Fit time          8.22    6.75    6.75    6.75    6.83    7.06    0.58    
Test time         0.38    0.37    0.38    0.17    0.18    0.30    0.10    


{'fit_time': (8.22192907333374,
  6.7509446144104,
  6.749925851821899,
  6.751333951950073,
  6.834836959838867),
 'test_mae': array([0.81773687, 0.82264968, 0.82248861, 0.8252913 , 0.8139703 ]),
 'test_rmse': array([1.01809564, 1.02506472, 1.02464407, 1.02339956, 1.01949683]),
 'test_time': (0.38004517555236816,
  0.37442588806152344,
  0.379197359085083,
  0.17391657829284668,
  0.18249225616455078)}

In [0]:
cross_validate(SVDpp(), data, cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0245  1.0291  1.0335  1.0199  1.0309  1.0276  0.0048  
MAE (testset)     0.8315  0.8314  0.8379  0.8275  0.8345  0.8325  0.0035  
Fit time          9.62    9.72    9.69    9.69    9.65    9.67    0.03    
Test time         0.44    0.24    0.26    0.24    0.24    0.28    0.08    


{'fit_time': (9.62392520904541,
  9.71738862991333,
  9.693756818771362,
  9.689836025238037,
  9.6489839553833),
 'test_mae': array([0.83145277, 0.8313558 , 0.83791351, 0.82747986, 0.83447873]),
 'test_rmse': array([1.02450308, 1.02913234, 1.03351589, 1.01986623, 1.03086907]),
 'test_time': (0.4440462589263916,
  0.23690438270568848,
  0.25811314582824707,
  0.23817801475524902,
  0.23901104927062988)}

In [0]:
cross_validate(KNNBaseline(), data, cv=5, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...


### Manually Splitting Test/Train (Exploration - Not Pursued)

In [0]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.25)

In [0]:
from surprise import accuracy

algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.0236


1.023587023218731

In [0]:
from surprise import KNNBasic
from surprise import Dataset

# Retrieve the trainset.
trainset = data.build_full_trainset()

# Build an algorithm, and train it.
algo = KNNBasic()
algo.fit(trainset)

Computing the msd similarity matrix...
